In [ ]:
from transformers import pipeline

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [ ]:
model = AutoModelForMaskedLM.from_pretrained('DeepPavlov/rubert-base-cased').to('cuda:0')
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
unmasker_ru = pipeline('fill-mask', model=model, tokenizer=tokenizer)

In [ ]:
import pandas as pd

In [ ]:
with open('more_cities_txt.txt', encoding='utf-8') as f:
    data1 = f.readlines()
with open('more_metals_txt.txt', encoding='utf-8') as f:
    data2 = f.readlines()
with open('more_names_txt.txt', encoding='utf-8') as f:
    data3 = f.readlines()
with open('more_people_txt.txt', encoding='utf-8') as f:
    data4 = f.readlines()

data = data1 + data2 + data3 + data4

In [ ]:
df = pd.DataFrame(columns=['sent','topic', 'masked', 'expect', 'pred1', 'pred2', 'pred3', 'pred4', 'pred5', 'similarity'])

In [ ]:
for d in data:
    res = unmasker_ru(d)
    info = d.split(';')
    sent = info[0]
    topic = info[1]
    masked = info[2]
    expect = info[3]
    pred1 = res[0]['token_str']
    pred2 = res[1]['token_str']
    pred3 = res[2]['token_str']
    pred4 = res[3]['token_str']
    pred5 = res[4]['token_str']
    df.loc[len(df.index )] = [sent, topic, masked, expect, pred1, pred2, pred3, pred4, pred5, 0]


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
df.to_excel('probed_data.xlsx', index=False)

In [ ]:
df.to_csv('probed_data.csv', index=False)

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
model = KeyedVectors.load_word2vec_format("/content/model.bin", binary=True)

In [ ]:
pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=20df0662cacf863ea525ef4286af17903736a2578bf58e596f32c0052097ca71
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from pymorphy2 import MorphAnalyzer

In [ ]:
morph = MorphAnalyzer()

In [ ]:
pos_comp = {'NOUN': 'NOUN',
            'ADJF': 'ADJ', 'ADJS': 'ADJ',
            'ADVB': 'ADV', 'COMP': 'ADV',
            'VERB': 'VERB'
            }

In [ ]:
graded_sim = pd.read_excel('/content/probed_data.xlsx')

In [ ]:
for index, row in graded_sim.iterrows ():  # по строчке в датасете

  expected = row['expect'][:-1].split('|')  # список ожиданий

  for i in range(len(expected)):
    e = expected[i].lower()  # к нижнему регистру
    e_norm = morph.parse(e)[0].normalized.word
    morph_id = pos_comp[str(morph.parse(e)[0].tag.POS)]  # замена тега
    expected[i] = e_norm + '_' + morph_id  # строка для симилярити

  similarities = []

  for pred in ['pred1','pred2', 'pred3', 'pred4', 'pred5']:
    sim_btw = 0

    p = row[pred].lower()  # предикт к нижнему регистру

    if p.isalpha() and str(morph.parse(p)[0].tag.POS) != 'None':
    # исключаем случаи "##" и "." и другую ерунду
      p1 = morph.parse(p)[0]
      pos = str(p1.normalized.tag.POS)  # тэг слова
      if pos in pos_comp.keys():
        morph_id = pos_comp[pos]  # новый тэг
        predict = morph.parse(p)[0].normalized.word + '_' + morph_id  # строка для симилярити
        if len(expected) == 2:
          try:
            sim_btw = max(model.similarity(expected[0], predict), model.similarity(expected[1], predict))
          except KeyError:
            sim_btw = 0
        else:
          try:
            sim_btw = model.similarity(expected[0], predict)
          except KeyError:
            sim_btw = 0
    else:
        sim_btw = 0
    similarities.append(sim_btw)
    sim_res = sum(similarities) / len(similarities)
    graded_sim.loc[index,'similarity'] = sim_res

In [ ]:
graded_sim.to_excel('graded_similarity_ff.xlsx', index=False)